<a href="https://colab.research.google.com/github/karimamr123/concept-project/blob/main/Evaluate_Llama2Base_model_for_LLM_res.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text2SQL Evaluate LLaMA2 base Model


In [ ]:
!pip install -q -U datasets
!pip install -q -U torch auto-gptq transformers optimum
!pip install -q -U peft trl einops accelerate xformers bitsandbytes
#! pip install -q -U rouge_score
#! pip install -q -U langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-

In [ ]:
! pip install torchvision==0.20.1+cu124 torch==2.5.1 torchaudio==2.5.1+cu124

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 399.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 6.2 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
  Attempting uninstall: torch
    Found existing installation: torch 2.6.0
    Uninstalling torch-2.6.0:
      Successfully uninstalled torch-2.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xformers 0.0.29.post2 requires torch==2.6.0, but you have torch 2.5.1 which is incompatible.


In [ ]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      Successfully uninstalled langchain-core-0.3.33
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.5
    Uninstalling langchain-text-splitters-0.3.5:
      Successfully uninstalled langchain-text-splitters-0.3.5
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.17
    Uninstalling langchain-0.3.17:
      Successfully uninstalled langchain-0.3.17


# Importing necessarry Libararies

In [ ]:
import pandas as pd
import json
import torch
import os
import time

# In case Login Required For Model
# from huggingface_hub import login
# from dotenv import load_dotenv

from datasets import load_dataset, Dataset,load_from_disk
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, AutoPeftModelForCausalLM
from transformers import GPTQConfig, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from trl import SFTTrainer
from time import perf_counter
from rich import print

# LangChain Imports
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate  #, LLMChain

# Imports for QA Retrieval Chain
from langchain.chains import RetrievalQA

# Import to Cleanup LLM Output
import textwrap

In [ ]:
# If Login Required for Model Access
# load_dotenv("/notebooks/.env")
# os.environ["TOKENIZERS_PARALLELISM"]="false"
# login(token=os.getenv("HUGGINGFACE_TOKEN"))

# Define the Model path and the dataset also paths on Drive

In [ ]:
model_id = "TheBloke/Llama-2-7B-GPTQ"
# model_id = "TheBloke/Llama-2-7b-Chat-GPTQ"

In [ ]:
# GDrive Location for Train/Test Data
DATA_PATH ="/content/drive/MyDrive/Data/"
DS_DIR = "sql_train_test"
PKL_DIR = "test/"
PKL_FILE ="sql_test.pkl"
LLMRES_PKL_FILE="sql_test_Llama2" # important this is the location of llm result change it if you divide the test # change it when re run it

### Let's define our helping Functions

In [ ]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

In [ ]:
def parse(text):
    start_marker = '### Response:'
    end_marker = '### End'
    start_index = text.find(start_marker)
    end_index = text.find(end_marker, start_index + len(start_marker))

    return (text[start_index + len(start_marker):].strip() if start_index != -1 and end_index == -1
            else text[start_index + len(start_marker):end_index].strip() if start_index != -1
            else None)

In [ ]:
def gen_eval_res(test_df):
  eval_res = []
  time_res = []
  row_no = 1
  # tmp = test_df.iloc[:2]
  for row in test_df.itertuples(index=False):
    start_time = perf_counter()
    # print(row.text)
    text = row.text # test_df['text'][ID]
    llm_res = pipe(text)[0]['generated_text']
    parse_res = parse(llm_res)
    end_time = perf_counter()
    eval_res.append(parse_res)
    time_res.append(end_time-start_time)
    if row_no % 10 == 0:
      print(f"*** Row {row_no} Done ***")
    row_no += 1

  test_df['eval_res'] = eval_res
  test_df['eval_time'] = time_res
  return test_df
  # display(pd.DataFrame(eval_res))
  # display(pd.DataFrame(time_res))

### let;s Load and Check Data on our drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load Test & Training Data from Disk
dataset = load_from_disk(DATA_PATH + DS_DIR)

In [ ]:
test_df = pd.read_pickle(DATA_PATH + PKL_DIR + PKL_FILE)

In [ ]:
display(dataset['train'])
display(dataset['test'])

Dataset({
    features: ['response', 'question', 'context', 'text', '__index_level_0__'],
    num_rows: 4086
})

Dataset({
    features: ['response', 'question', 'context', 'text', '__index_level_0__'],
    num_rows: 454
})

In [ ]:
display(test_df.head(2))
display(test_df.shape)

,response,question,context,__index_level_0__,text
0,SELECT College FROM match_season GROUP BY Coll...,Show the name of colleges that have at least t...,CREATE TABLE match_season (College VARCHAR),611,### Instruction:\n You are a powerful text-...
1,SELECT T1.name FROM Person AS T1 JOIN PersonFr...,Find the name of the person who has friends wi...,"CREATE TABLE Person (name VARCHAR, age INTEGER...",2556,### Instruction:\n You are a powerful text-...


(454, 5)

## Load Model from Hugging face

In [ ]:
quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True)

# Tokenizer - Base Model since we did not change it
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# LLM GPTQ Model
# To perform inference on the test dataset
model = AutoModelForCausalLM.from_pretrained(
          model_id,
          quantization_config=quantization_config_loading,
          device_map="auto"
        )

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/784 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:195: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.However, loading attributes (e.g. ['use_cuda_fp16', 'use_exllama', 'max_input_length', 'exllama_config', 'disable_exllama']) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.
  warnings.warn(warning_msg)
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:410: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:418: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='

model.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
Some weights of the model checkpoint at TheBloke/Llama-2-7B-GPTQ were not used when initializing LlamaForCausalLM: {'model.layers.30.self_attn.k_proj.bias', 'model.layers.20.mlp.down_proj.bias', 'model.layers.28.self_attn.v_proj.bias', 'model.layers.3.self_attn.o_proj.bias', 'model.layers.8.mlp.up_proj.bias', 'model.layers.12.self_attn.q_proj.bias', 'model.layers.18.mlp.gate_proj.bias', 'model.layers.27.self_attn.o_proj.bias', 'model.layers.27.self_attn.q_proj.bias', 'model.layers.31.mlp.up_proj.bias', 'model.layers.17.self_attn.q_proj.bias', 'model.layers.22.self_attn.v_proj.bias', 'model.layers.28.self_attn.k_proj.bias', 'model.layers.29.self_attn.v_proj.bias', 'model.layers.11.self_attn.k_proj.bias', 'model.layers.18.self_attn.q_proj.bias', 'model.layers.19.self_attn.k_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.31.self_attn.k_proj.bias', 'model.layers.1.mlp

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
# Get Model Memory Footprint = ~4GB
print(model.get_memory_footprint()/1e9) # GB

3.893862656

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=180,
    do_sample = True,
    temperature=0.1,
    top_k=4,
    # top_p=0.95,
    repetition_penalty=1.2
)
#  penalty_alpha=0.5,
#     # do_sample = True,
#     top_k=1,
#     # temperature=0.1,
#     repetition_penalty=1.2,
#     max_new_tokens=180

llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda:0
<ipython-input-18-04883f48fc6a>:19: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


# Check Model on Sample Test Question

In [ ]:
ID = 10

In [ ]:
test_df['text'][ID]

'### Instruction:\n    You are a powerful text-to-SQL model.     Your job is to answer questions about a database.     You are given a question and context regarding one or more tables.\n\n    You must output the SQL query that answers the question.\n\n    ### Input:\n    What is the average amount due for all the payments?\n    ### Context:\n    CREATE TABLE payments (amount_due INTEGER)\n    ### Response:\n    '

In [ ]:
display(test_df['response'][ID])

'SELECT AVG(amount_due) FROM payments'

In [ ]:
text = test_df['text'][ID]
llm_res = pipe(text)[0]['generated_text']
parse_res = parse(llm_res)
# print(llm_res)
print(parse_res)

SELECT AVG(amount_due) FROM payments;

### Get LLM Responses For All Questions

In [ ]:
eval_start_time = perf_counter()
test_LLM=gen_eval_res(test_df.iloc[201:300] ) # If the GPU disconnected you can use 100 row by 100 row like that test_df.iloc[:100] the next 100 row [100:201] and so on
eval_end_time = perf_counter()
print('Evaluation Took: ', round(eval_end_time-eval_start_time, 2))

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


*** Row 10 Done ***

*** Row 20 Done ***

*** Row 30 Done ***

*** Row 40 Done ***

...

*** Row 70 Done ***

*** Row 80 Done ***

*** Row 90 Done ***

<ipython-input-10-957ed58db131>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['eval_res'] = eval_res
<ipython-input-10-957ed58db131>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['eval_time'] = time_res


Evaluation Took:  4830.53

In [ ]:
test_LLM.to_pickle(DATA_PATH + PKL_DIR + LLMRES_PKL_FILE)

In [ ]:
# display(test_df['response'][:4].values)

# The last important thing save our LLM Results on Drive

In [ ]:
test_df2=pd.read_pickle(DATA_PATH + PKL_DIR + LLMRES_PKL_FILE)
#display the first 10 Rows
display(test_df2.head(10))

,response,question,context,__index_level_0__,text,eval_res,eval_time
201,SELECT Publisher FROM publication WHERE Price ...,Show the publishers that have publications wit...,"CREATE TABLE publication (Publisher VARCHAR, P...",132,### Instruction:\n You are a powerful text-...,SELECT Publisher FROM Publication WHERE Price ...,40.746721
202,SELECT COUNT(*) FROM catalog_contents,How many products are there in the records?,CREATE TABLE catalog_contents (Id VARCHAR),201,### Instruction:\n You are a powerful text-...,1098456732\n### Solution:\n```sql\nSELECT COUN...,23.232150
203,SELECT Name FROM teacher WHERE Age = 32 OR Age...,Show the name of teachers aged either 32 or 33?,"CREATE TABLE teacher (Name VARCHAR, Age VARCHAR)",4163,### Instruction:\n You are a powerful text-...,SELECT * FROM teacher WHERE age = '32' OR age=...,31.816946
204,SELECT catalog_entry_name FROM catalog_content...,Find the names of all the products whose stock...,CREATE TABLE catalog_contents (catalog_entry_n...,195,### Instruction:\n You are a powerful text-...,SELECT DISTINCT catalog_entry_name FROM catalo...,62.194320
205,"SELECT T1.Name, T2.Orchestra FROM conductor AS...",Show the names of conductors and the orchestra...,"CREATE TABLE conductor (Name VARCHAR, Conducto...",4420,### Instruction:\n You are a powerful text-...,SELECT Name FROM conductor WHERE Conductor_ID ...,90.863725
206,"SELECT t2.team_name, t2.ACC_Regular_Season FRO...",What is the team name and acc regular season s...,"CREATE TABLE university (school_id VARCHAR, fo...",544,### Instruction:\n You are a powerful text-...,100\n### Solution:\n```sql\nSELECT * FROM univ...,41.949336
207,SELECT COUNT(*) FROM ROLES,How many roles are there?,CREATE TABLE ROLES (Id VARCHAR),2429,### Instruction:\n You are a powerful text-...,10\n### Solution:\n```sql\nSELECT COUNT(*) FRO...,16.610504
208,SELECT T1.text_of_notes FROM Assessment_Notes ...,Find the texts of assessment notes for teacher...,CREATE TABLE Assessment_Notes (text_of_notes V...,1840,### Instruction:\n You are a powerful text-...,SELECT * FROM Assessment_Notes WHERE teacher_i...,15.860643
209,"SELECT T1.first_name, T2.department_name FROM ...",Display the first name and department name for...,CREATE TABLE departments (department_name VARC...,2024,### Instruction:\n You are a powerful text-...,"SELECT e.first_name AS FirstName, d.department...",33.220817
210,SELECT T2.f1 FROM user_profiles AS T1 JOIN fol...,Find the id of users who are followed by Mary ...,"CREATE TABLE follows (f1 VARCHAR, f2 VARCHAR);...",170,### Instruction:\n You are a powerful text-...,SELECT uid FROM user_profiles WHERE EXISTS(SEL...,49.051716


# You may save the test_LLM on several stages if i do so donot forget to change the file that the data saved on ok that what i mean >>LLMRES_PKL_FILE="sql_test_Llama2.pkl" change it to "sql_test_Llama2_v1.pkl" ,"sql_test_Llama2_v2.pkl" and so on according to how many times you will save it and finally concatenate them in a single dataframe  



In [ ]:
import os
print(os.path.exists(DATA_PATH + PKL_DIR + LLMRES_PKL_FILE))

True

In [ ]:
file_path = DATA_PATH + PKL_DIR + LLMRES_PKL_FILE
print("File exists:", os.path.exists(file_path))

File exists: True

In [ ]:
import shutil
shutil.copy("/content/drive/MyDrive/Data/test/sql_test_Llama2.pkl", "/content/sql_test_Llama2.pkl")


'/content/sql_test_Llama2.pkl'

In [ ]:
from google.colab import files
files.download("/content/sql_test_Llama2.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
eval_start_time = perf_counter()
test_LLM=gen_eval_res(test_df.iloc[300:399]) # If the GPU disconnected you can use 100 row by 100 row like that test_df.iloc[:100] the next 100 row [100:201] and so on
eval_end_time = perf_counter()
print('Evaluation Took: ', round(eval_end_time-eval_start_time, 2))

*** Row 10 Done ***

*** Row 20 Done ***

In [ ]:
test_LLM.to_pickle(DATA_PATH + PKL_DIR + LLMRES_PKL_FILE)

In [ ]:
test_df2=pd.read_pickle(DATA_PATH + PKL_DIR + LLMRES_PKL_FILE)
#display the first 10 Rows
display(test_df2.head(10))

NameError: name 'pd' is not defined

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# حدد المسار إلى الملف في Google Drive
file_drive_path = "/content/drive/MyDrive/Data/test/sql_test_Llama2.pkl"


Mounted at /content/drive


In [ ]:
import pickle

file_path_zeina = "A:\101-201\101-201\test-20250210T181556Z-001\test\sql_test_Llama2.pkl"
file_path_sara = "A:\1-100\1-100\test-20250210T181442Z-001\test\sql_test_Llama2.pkl"
# تحميل الملف من Google Drive
with open(file_drive_path, "rb") as f:
    list1 = pickle.load(f)

# تحميل الملفات المحلية
with open(file_path_zeina, "rb") as f:
    list2 = pickle.load(f)

with open(file_path_sara, "rb") as f:
    list3 = pickle.load(f)

# دمج القوائم
merged_list = list1 + list2 + list3

# حفظ القائمة المدمجة في ملف جديد
with open("merged_list.pkl", "wb") as f:
    pickle.dump(merged_list, f)

FileNotFoundError: [Errno 2] No such file or directory: 'A:A-201A-201\test-20250210T181556Z-001\test\\sql_test_Llama2.pkl'

In [ ]:
import os

file_path = r"A:\101-201\101-201\test-20250210T181556Z-001\test\sql_test_Llama2_sara.pkl"

if os.path.exists(file_path):
    print("✅ File found!")
else:
    print("❌ File not found:", file_path)

❌ File not found: A:\101-201\101-201\test-20250210T181556Z-001\test\sql_test_Llama2_sara.pkl


In [ ]:
import os
print(os.listdir())  # List all files in the current directory

['.config', 'drive', 'sample_data']


In [ ]:
import os

file_path = "/content/drive/MyDrive/pkl_files/sql_test_Llama2_sara.pkl"

if os.path.exists(file_path):
    print("✅ File found!")
else:
    print("❌ File not found:", file_path)

❌ File not found: /content/drive/MyDrive/pkl_files/sql_test_Llama2_sara.pkl


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os

file_path_zeina = "/content/drive/MyDrive/pkl_files/sql_test_Llama2_zeina.pkl"
file_path_zeina2 = "/content/drive/MyDrive/pkl_files/sql_test_Llama2_zeina2.pkl"
file_path_sara = "/content/drive/MyDrive/pkl_files/sql_test_Llama2_sara.pkl"

# Check if files exist
for file_path in [file_path_zeina, file_path_sara,file_path_zeina2]:
    if os.path.exists(file_path):
        print(f"✅ Found: {file_path}")
    else:
        print(f"❌ Not Found: {file_path}")

✅ Found: /content/drive/MyDrive/pkl_files/sql_test_Llama2_zeina.pkl
✅ Found: /content/drive/MyDrive/pkl_files/sql_test_Llama2_sara.pkl
✅ Found: /content/drive/MyDrive/pkl_files/sql_test_Llama2_zeina2.pkl


In [ ]:
file_drive_path = "/content/drive/MyDrive/Data/test/sql_test_Llama2.pkl"

import pickle

# Load the first file
with open(file_path_zeina, "rb") as f:
    data_zeina = pickle.load(f)


with open(file_path_zeina2, "rb") as f:
    data_zeina2 = pickle.load(f)

# Load the second file
with open(file_path_sara, "rb") as f:
    data_sara = pickle.load(f)

# Load the third file
with open(file_drive_path, "rb") as f:
    data_third = pickle.load(f)

# Merge (concatenate) the lists
merged_data = []
merged_data.extend(data_zeina)
merged_data.extend(data_zeina2)
merged_data.extend(data_sara)
merged_data.extend(data_third)

print(f"✅ Merged {len(data_zeina)} + {len(data_zeina2)} + {len(data_sara)} + {len(data_third)} rows successfully!")


✅ Merged 99 + 101 + 153 + 99 rows successfully!


In [ ]:
output_file = "/content/drive/MyDrive/sql_test_Llama2_mergedfinal.pkl"

with open(output_file, "wb") as f:
    pickle.dump(merged_data, f)

print(f"✅ Merged data saved as: {output_file}")

✅ Merged data saved as: /content/drive/MyDrive/sql_test_Llama2_mergedfinal.pkl


In [ ]:
merged_file_path = "/content/drive/MyDrive/sql_test_Llama2_mergedfinal.pkl"


In [ ]:
import pickle

# Load the merged file
with open(merged_file_path, "rb") as f:
    merged_data = pickle.load(f)

# Print the first 5 rows
print("✅ First 5 entries in the merged file:")
for i, entry in enumerate(merged_data[:5]):
    print(f"{i+1}: {entry}")

# Print total number of entries
print(f"\n✅ Total entries in the file: {len(merged_data)}")

✅ First 5 entries in the merged file:
1: response
2: question
3: context
4: __index_level_0__
5: text

✅ Total entries in the file: 28


In [ ]:
import pickle
import pandas as pd

# Load the file
with open(merged_file_path, "rb") as f:
    data = pickle.load(f)

# Check if it's a DataFrame
if isinstance(data, pd.DataFrame):
    print("✅ Data is a Pandas DataFrame")
    print(data.head())  # Show first 5 rows
else:
    print("✅ Data is a list/dictionary")
    print(data[:5])  # Print first 5 entries


✅ Data is a list/dictionary
['response', 'question', 'context', '__index_level_0__', 'text']


In [ ]:

df = pd.DataFrame(data)
print(df.head())  # Display first 5 rows

                   0
0           response
1           question
2            context
3  __index_level_0__
4               text


In [ ]:
print(type(data))


<class 'list'>


In [ ]:
print(data)


['response', 'question', 'context', '__index_level_0__', 'text', 'eval_res', 'eval_time', 'response', 'question', 'context', '__index_level_0__', 'text', 'eval_res', 'eval_time', 'response', 'question', 'context', '__index_level_0__', 'text', 'eval_res', 'eval_time', 'response', 'question', 'context', '__index_level_0__', 'text', 'eval_res', 'eval_time']


In [ ]:
import pickle

file_path = "/content/drive/MyDrive/pkl_files/sql_test_Llama2_zeina.pkl"

with open(file_path, "rb") as f:
    data = pickle.load(f)

print(type(data))  # Check if it's a list, dictionary, or DataFrame
print(data[:3])  # Print first 2 entries

<class 'pandas.core.frame.DataFrame'>
                                              response  \
101  SELECT name FROM airport WHERE name LIKE '%int...   
102  SELECT T1.first_name, T2.name FROM Owners AS T...   
103  SELECT DISTINCT T1.lname FROM Faculty AS T1 JO...   

                                              question  \
101  Which of the airport names contains the word '...   
102  List pairs of the owner's first name and the d...   
103  Find the first names of the faculty members wh...   

                                               context  __index_level_0__  \
101                CREATE TABLE airport (name VARCHAR)               3629   
102  CREATE TABLE Dogs (name VARCHAR, owner_id VARC...               4476   
103  CREATE TABLE activity (activity_name VARCHAR);...               3849   

                                                  text  \
101  ### Instruction:\n    You are a powerful text-...   
102  ### Instruction:\n    You are a powerful text-...   
103  ### Inst

In [ ]:
import pandas as pd

merged_data = pd.concat([data_zeina, data_zeina2, data_sara], ignore_index=True)

NameError: name 'data_zeina' is not defined

In [ ]:
merged_data = data_zeina + data_zeina2 + data_sara + data_third


NameError: name 'data_zeina' is not defined

In [ ]:
import pickle

# Define the file paths
file_drive_path = "/content/drive/MyDrive/Data/test/sql_test_Llama2.pkl"
file_path_zeina = "/content/drive/MyDrive/pkl_files/sql_test_Llama2_zeina.pkl"
file_path_zeina2 = "/content/drive/MyDrive/pkl_files/sql_test_Llama2_zeina2.pkl"
file_path_sara = "/content/drive/MyDrive/pkl_files/sql_test_Llama2_sara.pkl"

In [ ]:
def load_pickle(file_path):
    with open(file_path, 'rb') as f:
        return pickle.load(f)

# Load data from each file
data_drive = load_pickle(file_drive_path)
data_zeina = load_pickle(file_path_zeina)
data_zeina2 = load_pickle(file_path_zeina2)
data_sara = load_pickle(file_path_sara)

In [ ]:
concatenated_data = data_drive + data_zeina + data_zeina2 + data_sara

In [ ]:
output_file_path = "/content/drive/MyDrive/Data/test/concatenated_sql_test_Llama2.pkl"
with open(output_file_path, 'wb') as f:
    pickle.dump(concatenated_data, f)

print("Data concatenated and saved to:", output_file_path)

Data concatenated and saved to: /content/drive/MyDrive/Data/test/concatenated_sql_test_Llama2.pkl


In [ ]:
with open(output_file_path, 'rb') as f:
    loaded_data = pickle.load(f)

print("✅ Loaded concatenated data successfully!")
print("First 5 entries:\n", loaded_data[:5])  # Adjust based on your data type

✅ Loaded concatenated data successfully!
First 5 entries:
     response question context  __index_level_0__ text eval_res  eval_time
100      NaN      NaN     NaN                NaN  NaN      NaN        NaN
101      NaN      NaN     NaN                NaN  NaN      NaN        NaN
102      NaN      NaN     NaN                NaN  NaN      NaN        NaN
103      NaN      NaN     NaN                NaN  NaN      NaN        NaN
104      NaN      NaN     NaN                NaN  NaN      NaN        NaN


In [ ]:
print(loaded_data.head())  # Print first 5 rows


    response question context  __index_level_0__ text eval_res  eval_time
100      NaN      NaN     NaN                NaN  NaN      NaN        NaN
101      NaN      NaN     NaN                NaN  NaN      NaN        NaN
102      NaN      NaN     NaN                NaN  NaN      NaN        NaN
103      NaN      NaN     NaN                NaN  NaN      NaN        NaN
104      NaN      NaN     NaN                NaN  NaN      NaN        NaN


In [ ]:
print("Columns in data_drive:", data_drive.columns)
print("Columns in data_zeina:", data_zeina.columns)
print("Columns in data_zeina2:", data_zeina2.columns)
print("Columns in data_sara:", data_sara.columns)

Columns in data_drive: Index(['response', 'question', 'context', '__index_level_0__', 'text',
       'eval_res', 'eval_time'],
      dtype='object')
Columns in data_zeina: Index(['response', 'question', 'context', '__index_level_0__', 'text',
       'eval_res', 'eval_time'],
      dtype='object')
Columns in data_zeina2: Index(['response', 'question', 'context', '__index_level_0__', 'text',
       'eval_res', 'eval_time'],
      dtype='object')
Columns in data_sara: Index(['response', 'question', 'context', '__index_level_0__', 'text',
       'eval_res', 'eval_time'],
      dtype='object')


In [ ]:
print(data_drive.dtypes)
print(data_zeina.dtypes)
print(data_zeina2.dtypes)
print(data_sara.dtypes)


response              object
question              object
context               object
__index_level_0__      int64
text                  object
eval_res              object
eval_time            float64
dtype: object
response              object
question              object
context               object
__index_level_0__      int64
text                  object
eval_res              object
eval_time            float64
dtype: object
response              object
question              object
context               object
__index_level_0__      int64
text                  object
eval_res              object
eval_time            float64
dtype: object
response              object
question              object
context               object
__index_level_0__      int64
text                  object
eval_res              object
eval_time            float64
dtype: object


In [ ]:
concatenated_data = pd.concat([data_drive, data_zeina, data_zeina2, data_sara], ignore_index=True)


In [ ]:
print(concatenated_data.isna().sum())  # Count NaNs per column


response             0
question             0
context              0
__index_level_0__    0
text                 0
eval_res             0
eval_time            0
dtype: int64


    response question context  __index_level_0__ text eval_res  eval_time
100      NaN      NaN     NaN                NaN  NaN      NaN        NaN
101      NaN      NaN     NaN                NaN  NaN      NaN        NaN
102      NaN      NaN     NaN                NaN  NaN      NaN        NaN
103      NaN      NaN     NaN                NaN  NaN      NaN        NaN
104      NaN      NaN     NaN                NaN  NaN      NaN        NaN


In [ ]:
print(concatenated_data.head(10))  # First 10 rows
print(concatenated_data.tail(10))  # Last 10 rows

                                            response  \
0  SELECT Publisher FROM publication WHERE Price ...   
1              SELECT COUNT(*) FROM catalog_contents   
2  SELECT Name FROM teacher WHERE Age = 32 OR Age...   
3  SELECT catalog_entry_name FROM catalog_content...   
4  SELECT T1.Name, T2.Orchestra FROM conductor AS...   
5  SELECT t2.team_name, t2.ACC_Regular_Season FRO...   
6                         SELECT COUNT(*) FROM ROLES   
7  SELECT T1.text_of_notes FROM Assessment_Notes ...   
8  SELECT T1.first_name, T2.department_name FROM ...   
9  SELECT T2.f1 FROM user_profiles AS T1 JOIN fol...   

                                            question  \
0  Show the publishers that have publications wit...   
1        How many products are there in the records?   
2    Show the name of teachers aged either 32 or 33?   
3  Find the names of all the products whose stock...   
4  Show the names of conductors and the orchestra...   
5  What is the team name and acc regular season

In [ ]:
output_file_path = "/content/drive/MyDrive/Data/test/concatenated_sql_test_Llama2.pkl"
concatenated_data.to_pickle(output_file_path)

# Reload and print again
reloaded_data = pd.read_pickle(output_file_path)
print(reloaded_data.head(10))

                                            response  \
0  SELECT Publisher FROM publication WHERE Price ...   
1              SELECT COUNT(*) FROM catalog_contents   
2  SELECT Name FROM teacher WHERE Age = 32 OR Age...   
3  SELECT catalog_entry_name FROM catalog_content...   
4  SELECT T1.Name, T2.Orchestra FROM conductor AS...   
5  SELECT t2.team_name, t2.ACC_Regular_Season FRO...   
6                         SELECT COUNT(*) FROM ROLES   
7  SELECT T1.text_of_notes FROM Assessment_Notes ...   
8  SELECT T1.first_name, T2.department_name FROM ...   
9  SELECT T2.f1 FROM user_profiles AS T1 JOIN fol...   

                                            question  \
0  Show the publishers that have publications wit...   
1        How many products are there in the records?   
2    Show the name of teachers aged either 32 or 33?   
3  Find the names of all the products whose stock...   
4  Show the names of conductors and the orchestra...   
5  What is the team name and acc regular season

In [ ]:
print(concatenated_data[['response', 'question', 'context']].head(10))



                                            response  \
0  SELECT Publisher FROM publication WHERE Price ...   
1              SELECT COUNT(*) FROM catalog_contents   
2  SELECT Name FROM teacher WHERE Age = 32 OR Age...   
3  SELECT catalog_entry_name FROM catalog_content...   
4  SELECT T1.Name, T2.Orchestra FROM conductor AS...   
5  SELECT t2.team_name, t2.ACC_Regular_Season FRO...   
6                         SELECT COUNT(*) FROM ROLES   
7  SELECT T1.text_of_notes FROM Assessment_Notes ...   
8  SELECT T1.first_name, T2.department_name FROM ...   
9  SELECT T2.f1 FROM user_profiles AS T1 JOIN fol...   

                                            question  \
0  Show the publishers that have publications wit...   
1        How many products are there in the records?   
2    Show the name of teachers aged either 32 or 33?   
3  Find the names of all the products whose stock...   
4  Show the names of conductors and the orchestra...   
5  What is the team name and acc regular season

In [ ]:
output_pkl_path = "/content/drive/MyDrive/Data/test/concatenated_sql_test_Llama2.pkl"
concatenated_data.to_pickle(output_pkl_path)

print("✅ Data saved successfully as Pickle:", output_pkl_path)

✅ Data saved successfully as Pickle: /content/drive/MyDrive/Data/test/concatenated_sql_test_Llama2.pkl


In [ ]:
import pandas as pd

# Load the concatenated pickle file
output_pkl_path = "/content/drive/MyDrive/Data/test/concatenated_sql_test_Llama2.pkl"
df = pd.read_pickle(output_pkl_path)

# Display first few rows
print(df.head())

                                            response  \
0  SELECT Publisher FROM publication WHERE Price ...   
1              SELECT COUNT(*) FROM catalog_contents   
2  SELECT Name FROM teacher WHERE Age = 32 OR Age...   
3  SELECT catalog_entry_name FROM catalog_content...   
4  SELECT T1.Name, T2.Orchestra FROM conductor AS...   

                                            question  \
0  Show the publishers that have publications wit...   
1        How many products are there in the records?   
2    Show the name of teachers aged either 32 or 33?   
3  Find the names of all the products whose stock...   
4  Show the names of conductors and the orchestra...   

                                             context  __index_level_0__  \
0  CREATE TABLE publication (Publisher VARCHAR, P...                132   
1         CREATE TABLE catalog_contents (Id VARCHAR)                201   
2   CREATE TABLE teacher (Name VARCHAR, Age VARCHAR)               4163   
3  CREATE TABLE catalog_co

In [ ]:
import pandas as pd

# Load the concatenated pickle file
output_pkl_path = "/content/drive/MyDrive/Data/test/concatenated_sql_test_Llama2.pkl"
df = pd.read_pickle(output_pkl_path)

# Read rows from index 100 to 120
subset_df = df.iloc[100:121]  # 121 to include row 120

# Display the extracted rows
print(subset_df)


                                              response  \
100  SELECT T1.first_name, T2.name FROM Owners AS T...   
101  SELECT DISTINCT T1.lname FROM Faculty AS T1 JO...   
102  SELECT t2.product_details, t2.product_id FROM ...   
103  SELECT name, distance FROM Aircraft WHERE aid ...   
104       SELECT Name FROM editor ORDER BY Age LIMIT 1   
105  SELECT grade, COUNT(*) FROM Highschooler GROUP...   
106   SELECT weight FROM pets ORDER BY pet_age LIMIT 1   
107  SELECT model_name FROM chip_model ORDER BY lau...   
108  SELECT billing_state, COUNT(*) FROM invoices W...   
109  SELECT apt_type_code, COUNT(*) FROM Apartments...   
110                    SELECT Name, Arrival FROM train   
111  SELECT T2.Store_Email_Address FROM Addresses A...   
112  SELECT First_year, Last_year FROM party WHERE ...   
113  SELECT T1.Title FROM book AS T1 JOIN publicati...   
114  SELECT COUNT(DISTINCT payment_method) FROM cus...   
115  SELECT Status FROM roller_coaster GROUP BY Sta...   
116  SELECT De